# Loop 4 LB Feedback Analysis

## Situation
- 4 submissions, all scoring 70.6473 (CV = LB exactly)
- Target: 68.919154
- Gap: 1.728 points (2.45%)

## Key Findings
1. C++ SA optimizer found 0 improvements after 4 generations
2. Corner extraction found 0 improvements
3. All public solutions are at the same local optimum
4. SA-based optimization cannot close the gap

## What We Need
Fundamentally different approaches, not more optimization of the same solutions.

In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import matplotlib.pyplot as plt

getcontext().prec = 30
scale_factor = 1

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(Decimal('0.0') * scale_factor), float(tip_y * scale_factor)),
            (float(top_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
            (float(top_w / Decimal('4') * scale_factor), float(tier_1_y * scale_factor)),
            (float(mid_w / Decimal('2') * scale_factor), float(tier_2_y * scale_factor)),
            (float(mid_w / Decimal('4') * scale_factor), float(tier_2_y * scale_factor)),
            (float(base_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(base_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(mid_w / Decimal('4')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(mid_w / Decimal('2')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(top_w / Decimal('4')) * scale_factor), float(tier_1_y * scale_factor)),
            (float(-(top_w / Decimal('2')) * scale_factor), float(tier_1_y * scale_factor)),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def load_solution(csv_path):
    df = pd.read_csv(csv_path)
    df['x'] = df['x'].astype(str).str.strip().str.lstrip('s')
    df['y'] = df['y'].astype(str).str.strip().str.lstrip('s')
    df['deg'] = df['deg'].astype(str).str.strip().str.lstrip('s')
    df[['group_id', 'item_id']] = df['id'].str.split('_', n=2, expand=True)
    
    solution = {}
    for group_id, group_data in df.groupby('group_id'):
        n = int(group_id)
        trees = [(row['x'], row['y'], row['deg']) for _, row in group_data.iterrows()]
        solution[n] = trees
    
    return solution

def score_config(trees_data):
    tree_list = [ChristmasTree(x, y, deg) for x, y, deg in trees_data]
    side = get_tree_list_side_length(tree_list)
    n = len(trees_data)
    return side ** 2 / n

print('Functions defined.')

In [ ]:
# Load current best solution
solution = load_solution('/home/submission/submission.csv')

# Calculate per-N scores
scores = {n: score_config(solution[n]) for n in range(1, 201)}
total_score = sum(scores.values())
print(f'Current total score: {total_score:.6f}')
print(f'Target: 68.919154')
print(f'Gap: {total_score - 68.919154:.6f} ({(total_score - 68.919154) / 68.919154 * 100:.2f}%)')

In [ ]:
# Analyze where the gap is coming from
# Calculate theoretical minimum for each N (assuming perfect packing)
# For N trees, theoretical minimum side = sqrt(N * tree_area)
# Tree area is approximately 0.35 * 1.0 = 0.35 (rough estimate)

# Let's look at efficiency: actual_score / theoretical_minimum
# Higher efficiency = closer to optimal

# First, let's understand the tree dimensions
tree = ChristmasTree('0', '0', '0')
bounds = tree.polygon.bounds
tree_width = bounds[2] - bounds[0]
tree_height = bounds[3] - bounds[1]
tree_area = tree.polygon.area
print(f'Tree dimensions: width={tree_width:.4f}, height={tree_height:.4f}')
print(f'Tree area: {tree_area:.4f}')
print(f'Bounding box area: {tree_width * tree_height:.4f}')
print(f'Fill ratio: {tree_area / (tree_width * tree_height):.4f}')

In [ ]:
# Calculate efficiency for each N
# Theoretical minimum score = tree_area (if trees could pack perfectly)
# Actual score = side^2 / N
# Efficiency = tree_area / actual_score

efficiencies = {}
for n in range(1, 201):
    actual_score = scores[n]
    theoretical_min = tree_area  # Perfect packing would give this
    efficiency = theoretical_min / actual_score
    efficiencies[n] = efficiency

# Find N values with worst efficiency (most room for improvement)
worst_efficiency = sorted(efficiencies.items(), key=lambda x: x[1])[:20]
print('N values with worst efficiency (most room for improvement):')
for n, eff in worst_efficiency:
    print(f'  N={n:3d}: efficiency={eff:.4f}, score={scores[n]:.6f}, contribution={scores[n]:.4f}')

In [ ]:
# Calculate how much improvement we need per N to reach target
target = 68.919154
gap = total_score - target
print(f'Total gap: {gap:.6f}')
print(f'Average gap per N: {gap / 200:.6f}')

# If we improve each N by the same percentage, what percentage is needed?
required_improvement_pct = 1 - (target / total_score)
print(f'Required improvement: {required_improvement_pct * 100:.2f}%')

# What if we focus on specific N values?
# Let's see which N values contribute most to the total score
contributions = sorted([(n, scores[n]) for n in range(1, 201)], key=lambda x: -x[1])
print('\nTop 20 N values by score contribution:')
for n, score in contributions[:20]:
    print(f'  N={n:3d}: score={score:.6f}')

In [ ]:
# Key insight: Small N values have the highest per-N scores
# N=1 is already optimal (45 degrees)
# N=2-10 have the most room for improvement

# Let's visualize the current solution for N=2 to understand the pattern
def plot_trees(n, trees_data, title=''):
    fig, ax = plt.subplots(figsize=(8, 8))
    for x, y, deg in trees_data:
        tree = ChristmasTree(x, y, deg)
        xs, ys = tree.polygon.exterior.xy
        ax.fill(xs, ys, alpha=0.5)
        ax.plot(xs, ys, 'k-', linewidth=0.5)
    ax.set_aspect('equal')
    ax.set_title(f'{title}\nN={n}, Score={score_config(trees_data):.6f}')
    plt.show()

for n in [2, 3, 4, 5, 6]:
    plot_trees(n, solution[n], f'Current solution for N={n}')

In [ ]:
# The egortrushin kernel uses tessellation approach:
# Create grid patterns with specific nx, ny translations
# This is fundamentally different from SA optimization

# Let's try to understand the tessellation approach
# For N=nx*ny trees, create a grid with nx columns and ny rows
# Each tree is translated by (dx, dy) from the previous one

# Key insight from egortrushin:
# - For N=72 (8x9), N=100 (10x10), N=110 (5x11), etc.
# - The grid pattern can be more efficient than random packing

# Let's check which N values are perfect rectangles
rectangular_n = []
for n in range(1, 201):
    for nx in range(1, n+1):
        if n % nx == 0:
            ny = n // nx
            if nx <= ny:  # Avoid duplicates
                rectangular_n.append((n, nx, ny))

print('N values that can be arranged as rectangles:')
for n, nx, ny in rectangular_n[:30]:
    print(f'  N={n:3d}: {nx}x{ny}')

In [ ]:
# The key insight is that tessellation can find better solutions for specific N values
# But we need to understand WHICH N values have room for improvement

# Let's compare our scores to the theoretical minimum more carefully
# The theoretical minimum for N trees is: (sqrt(N) * tree_side)^2 / N = tree_side^2
# But this assumes perfect square packing, which isn't possible for trees

# A better estimate: for N trees in a grid, the minimum side is approximately:
# side = max(nx * dx, ny * dy) where dx, dy are the minimum spacing

# Let's estimate the minimum spacing by looking at N=1
n1_score = scores[1]
n1_side = np.sqrt(n1_score)  # side^2 / 1 = side^2
print(f'N=1 side: {n1_side:.6f}')
print(f'N=1 score: {n1_score:.6f}')

# For N=2, the minimum side should be approximately 2 * tree_width (horizontal) or tree_height (vertical)
# Let's see what we have
n2_score = scores[2]
n2_side = np.sqrt(n2_score * 2)
print(f'N=2 side: {n2_side:.6f}')
print(f'N=2 score: {n2_score:.6f}')

# Efficiency ratio
print(f'N=2 efficiency vs N=1: {n1_score / n2_score:.4f}')

In [ ]:
# Let's look at the leaderboard to understand what's possible
# The target is 68.919154, which is 2.45% better than our 70.647306

# If we assume the improvement is distributed proportionally across all N values,
# we need each N to improve by 2.45%

# But the improvement might be concentrated in specific N values
# Let's calculate what scores we'd need for each N to reach the target

required_scores = {}
for n in range(1, 201):
    # If we improve each N by the same percentage
    required_scores[n] = scores[n] * (target / total_score)

print('Required scores to reach target (proportional improvement):')
for n in [1, 2, 3, 4, 5, 10, 20, 50, 100, 200]:
    print(f'  N={n:3d}: current={scores[n]:.6f}, required={required_scores[n]:.6f}, gap={scores[n] - required_scores[n]:.6f}')

In [ ]:
# Summary of findings:
# 1. All 4 submissions score exactly 70.6473 (CV = LB)
# 2. SA optimization cannot escape the local optimum
# 3. The gap to target is 1.728 points (2.45%)
# 4. Small N values (1-10) contribute most per-N
# 5. N=1 is already optimal (45 degrees)
# 6. The tessellation approach from egortrushin is fundamentally different

# Key strategies to try:
# 1. Tessellation-based construction for specific N values (rectangular N)
# 2. Genetic algorithms with diverse initial populations
# 3. Constraint programming / integer programming
# 4. Focus on small N (2-20) where improvements have most impact

print('\n=== STRATEGIC RECOMMENDATIONS ===')
print('1. Implement tessellation approach from egortrushin kernel')
print('2. Focus on small N (2-20) where improvements have most impact')
print('3. Try genetic algorithms with diverse initial populations')
print('4. Consider asymmetric packing (mentioned in discussions)')